In [1]:
import rdflib
from nilearn import datasets

/home/gzanitti/miniconda3/envs/neurolang/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
from rdflib.namespace import RDF, RDFS, OWL, SKOS
from rdflib import BNode


In [140]:
rdfGraph = rdflib.Graph()
rdfGraph.load(cogAt_old)

In [65]:
classes = set()

for s, _, _ in rdfGraph.triples((None, RDF.type , OWL.Class)):
    if not isinstance(s, BNode):
        classes.add(s)
for s, _, _ in rdfGraph.triples((None, RDF.type , RDFS.Class)):
    if not isinstance(s, BNode):
        classes.add(s)

#for s, _, o in rdfGraph.triples((None, RDFS.subClassOf , None)):
#    if s not in classes and not isinstance(s, BNode):
#        classes.add(s)
#    if o not in classes and not isinstance(o, BNode):
#        classes.add(o)

#for _, _, o in rdfGraph.triples((None, RDFS.domain , None)):
#    if o not in classes and not isinstance(o, BNode):
#        classes.add(o)
#for _, _, o in rdfGraph.triples((None, RDFS.range , None)):
#    if o not in classes and not isinstance(o, BNode):
#        classes.add(o)

In [66]:
len(classes)

199

In [79]:
def _parseSubClass(a, b, c):
    if isinstance(c, BNode):
        bnode_triples = list(rdfGraph.triples((c, None, None)))
        restriction_dic = {b:c for _, b, c in bnode_triples}
        if OWL.Restriction in restriction_dic.values():
            _parse_restriction(restriction_dic)
        elif OWL.intersectionOf in restriction_dic.keys():
            c = restriction_dic[OWL.intersectionOf]
            bnode_triples = list(rdfGraph.triples((c, None, None)))
            print(bnode_triples)
            raise NotImplementedError('intersectionOf')
        elif OWL.unionOf in restriction_dic.keys():
            raise NotImplementedError('unionOf')
        elif OWL.complementOf in restriction_dic.keys():
            raise NotImplementedError('complementOf')
        else:
            print('Something not parsed here', restriction_dic)
    else:
        return ('subClass', a, c)
        
def _parse_restriction(restriction_dic):
    prop = restriction_dic[OWL.onProperty]
    
    if OWL.someValuesFrom in restriction_dic.keys():
        node = restriction_dic[OWL.someValuesFrom]
        if isinstance(node, BNode):
            node = _solve_BNode(node)
        else:
            node = [node]
        _parse_someValuesFrom(prop, node)
    
    elif OWL.allValuesFrom in restriction_dic.keys():
        node = restriction_dic[OWL.allValuesFrom]
        if isinstance(node, BNode):
            node = _solve_BNode(node)
        else:
            node = [node]
        _parse_allValuesFrom(prop, node)
    
    elif OWL.hasValue in restriction_dic.keys():
        node = restriction_dic[OWL.hasValue]
        if isinstance(node, BNode):
            node = _solve_BNode(node)
        else:
            node = [node]
        _parse_hasValue(prop, node)
    
    elif OWL.minCardinality in restriction_dic.keys():
        print('minCardinality constraints cannot be implemented in Datalog syntax')
        
    elif OWL.maxCardinality in restriction_dic.keys():
        print('maxCardinality constraints cannot be implemented in Datalog syntax')
    
    else:
        print('Restriction cannot be parsed:', restriction_dic.keys())
    
def _parse_someValuesFrom(prop, nodes):
    res = []
    for n in nodes:
        res.append((prop, n))
        
    return res
        
def _parse_allValuesFrom(prop, nodes):
    res = []
    for n in nodes:
        res.append((prop, n))
        
    return res
        
def _parse_hasValue(prop, nodes):
    pass
    #for n in nodes:
    #    print('[x] Parseo hasValue', prop, n)
        
def _solve_BNode(initial_node):
    print('Nodo a resolver', initial_node)

    list_node = RDF.nil
    values = []
    for node_triples in rdfGraph.triples((initial_node, None, None)):
        if OWL.unionOf == node_triples[1]:
            list_node = node_triples[2]

    while list_node != RDF.nil and list_node is not None:
        list_iter = rdfGraph.triples((list_node, None, None))
        list_iter = list(list_iter)
        values.append(_get_list_first_value(list_iter))
        list_node = _get_list_rest_value(list_iter)

    print(values)
    return values
        
def _get_list_first_value(list_iter):
    for triple in list_iter:
        if RDF.first == triple[1]:
            return triple[2]
        
def _get_list_rest_value(list_iter):
    for triple in list_iter:
        if RDF.rest == triple[1]:
            return triple[2]
    
def _parseLabel(a, b, c):
    pass
    #print('[x] Parseo label', c)
    
def _parsePrefLabel(a, b, c):
    pass
    #print('[x] Parseo prefLabel', c)
    
def _parseAltLabel(a, b, c):
    pass
    #print('[x] Parseo altLabel', c)
    
def _parseHasTopConcept(a, b, c):
    pass
    #print('[x] Parseo hasTopConcept', c)
    
def _parseEnumeratedClass(a, b, c):
    raise NotImplementedError

def _parseEquivalentClass(a, b, c):
    pass
    #print('EquivalentClass, no implementable')
    # No se puede implementar en DL
    # <->

def _parseDisjointClass(a, b,  c):
    #inconsistent(C, D) :− disjointWith(C, D), type(X, C), type(X, D).
    raise NotImplementedError

def _parseDatatype(a, b, c):
    #Se usa como definicion de tipos, ver mas
    raise NotImplementedError
    
def _parseProperty(a, b, c):
    pass
    #print('[x] Parseo propiedad general', b, c)

/home/gzanitti/miniconda3/envs/neurolang/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
for v, clase in enumerate(list(classes)):
    print(v, clase)
    print()
    for a, b, c in rdfGraph.triples((clase, None, None)):
        
        if b == RDF.type and c == OWL.Class:
            # The triple used to index the class must be skipped.
            print('[x] Skyped', c)
        
        elif b == RDFS.subClassOf:
            _parseSubClass(a, b, c)
        
        elif b == RDFS.label:
            _parseLabel(a, b, c)
            
        elif b == OWL.equivalentClass:
            _parseEquivalentClass(a, b,  c)
            
        elif b == OWL.EnumeratedClass:
            _parseEnumeratedClass(a, b, c)

        elif b == OWL.DisjointClass:
            _parseDisjointClass(a, b, c)
        
        elif b == SKOS.prefLabel:
            _parsePrefLabel(a, b, c)

        elif b == SKOS.hasTopConcept:
            _parseHasTopConcept(a, b, c)

        elif b == SKOS.altLabel:
            _parseAltLabel(a, b, c)
            
        else:
            if not isinstance(c, BNode):
                _parseProperty(a, b, c)
            else:
                raise NotImplementedError

    print('_________________')
    print()
    

0 http://www.cogpo.org/ontologies/COGPO_00032

[x] Skyped http://www.w3.org/2002/07/owl#Class
_________________

1 http://www.cogpo.org/ontologies/CogPOver1.owl#COGPO_00069

[x] Skyped http://www.w3.org/2002/07/owl#Class
_________________

2 http://www.cogpo.org/ontologies/CogPOver1.owl#COGPO_00083

[x] Skyped http://www.w3.org/2002/07/owl#Class
_________________

3 http://www.cogpo.org/ontologies/COGPO_00027

[x] Skyped http://www.w3.org/2002/07/owl#Class
_________________

4 http://purl.org/obo/owl/FMA#FMA_9664

[x] Skyped http://www.w3.org/2002/07/owl#Class
_________________

5 http://www.cogpo.org/ontologies/CogPOver1.owl#Naming_(Covert)_Paradigm

[x] Skyped http://www.w3.org/2002/07/owl#Class
_________________

6 http://www.cogpo.org/ontologies/CogPOver1.owl#COGPO_00049

[x] Skyped http://www.w3.org/2002/07/owl#Class
_________________

7 http://www.cogpo.org/ontologies/COGPO_00150

[x] Skyped http://www.w3.org/2002/07/owl#Class
_________________

8 http://www.cogpo.org/ontologies/

NotImplementedError: intersectionOf